<a href="https://colab.research.google.com/github/MarkNCI/MLAnalytics_Exp/blob/main/Thesis_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers -q
!pip install streamlit -q
!pip install PyMuPDF -q

In [8]:
!streamlit --version

Streamlit, version 1.36.0


In [4]:
import pandas as pd
import pymupdf
# import spacy

In [5]:
file_name = '/content/Operational_range_bounding_of_spectroscopy_models_with_anomaly_detection_draft.pdf'

In [6]:
def pdf_to_dataframe(input_file):
  metadata = []
  pdf = pymupdf.open(input_file)
  # Get page content as dict and blocks of data
  for page in pdf:
    pages = page.get_text('dict')
    blocks = pages['blocks']
    # Extract metadata from blocks and filter for text, font size and font name
    for block in blocks:
      if 'lines' in block.keys():
        spans = block['lines']
        for span in spans:
          data = span['spans']
          for x in data:
            metadata.append((x['text'], x['size'], x['font']))
  pdf.close()
  df = pd.DataFrame(metadata,columns = ['text','size','font'])
  return df

In [7]:
df = pdf_to_dataframe(file_name)
print(df['font'].value_counts())

font
Kp-Regular              468
DejaVuSans              156
Kp-Medium                63
Kp-Italic                55
Kp-Expert-Regular        26
Kp--M-Italic             22
DejaVuSans-Bold          12
DejaVuSans-Oblique        7
Kp--M-Sy-Regular          5
Tt-Kp-Regular             4
Kp-Companion-Regular      2
Cmsy10                    2
Kp-Expert-Medium          1
Kp--M-Ex-Regular          1
MSBM10                    1
Kp-SmallCaps-Regular      1
Name: count, dtype: int64


In [8]:
# Kp-Regular holds actual text so filter for that
df = df.loc[df['font'] == 'Kp-Regular']
# Column to list for preprocessing
corpus = ' '.join(df['text'])
corpus

'DRAFT version. Last modified: 2024-05-30 22:03:17Z Operational range bounding of spectroscopy models with anomaly detection Luís F. Simões 1 , Pierluigi Casale 1 , Marília Felismino 1 , Kai Hou Yip 2 , Ingo P. Waldmann 2 , Giovanna Tinetti 2 , Theresa Lueftinger 3 1 2 3 As Machine Learning adoption grows across risk- averse industries such as aerospace, various e orts are underway to design the validation processes that will ensure safety in complex operational settings [1–5].  being advocated by such e orts consists of a safety mechanism that continu- ously monitors system observables, and intervenes if it judges the model will exceed the bounds of a safe domain for a particular input. Unlike monitor- ing systems that run asynchronously from the pre- diction model and perform evaluations over assem- bled production data [6], the safety cage architecture runs a monitoring system in parallel with the predic- tion model, and operates at an individual sample level. This places explicit b

In [9]:
# from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer

In [10]:
import textwrap
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

inputs = tokenizer.encode("summarize: " + corpus, return_tensors="pt", max_length=1024, truncation=True, padding='longest') # Added Padding
summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
formatted_summary = "\n".join(textwrap.wrap(summary, width=80))

In [24]:
text_list = ["The Ariel Data Challenge [9–11] is a series of data competitions organised by mission scientists and engineers within the Ariel Consortium. The 2019 and 2021 editions of the challenge looked for innovative detrending solutions to extract planetary spectra from observed spectroscopic light curves. The simulations can however never represent the actual data, which will su er from unknown unknowns. Ensuring the reliable operation of Machine Learning models, as they move from simulated to real data, with unknowable ground truths, is of paramount importance. We conduct our experiments over the datasets from the 2019 and 2019 Ariel Data Challenges, hereby designated as ADC19 and ADC21 [9]. DRAFT version. Last modified: 2024-05-30 22:03:17Z  Transit light curves for the same star/planet system, produced by the simulation pipelines [12–15] of the ADC19 or ADC21 challenges. We treat the photon noise instances of the star’s first stellar spot configuration as repeated observations and aggregate them across the same system. The data process simulated 10 stellar spot configurations, which have to be identified and corrected in the derivation of the derivations. We use the train data that was made for those competitions. Experiments are restricted to a core of 1468 and 1256 star/Planet systems, respectively, that were simulated in both datasets, with the sameStar/Planet parameters. We treated the photon Noise instances as repeated Observations of the same System, and corrected them in the derived derivation. The results were published in the journal The Astrophysical Journal [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 59,. 60, 61, 62, 63, 64, 63,. 64, 65, 68, 69, 70, 74, 75, 80, 78, 81, 82, 83, 84, 84,. 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 94,. 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 116, 113,. 111, 114,. 113, 116,. 116, 117, 118, 119, 113 and 114, 118,. 119, 120, 123, 123,. 130, 130, 116 and 123, 124, 129, 130,. 131, 128, 131, 134, 135, 135,. 131,. 130,. 135, 134,. 137, 137, 138, 139, 140, 145, 149, 151, 152, 163, 164, 165, 168, 169, 173, 174, 175, 178, 179, 194, 199, 204, 199,. 207, 204,. 204, 205, 206, 207, 208, 209, 210, 217, 209,. 208, 204. 211, 209. 204, 215, 215,. 216, 217,. 217, 208,. 215, 217. 224, 215. 226, 215., 216, 209., 215, 216, 220, 217; 217, 224, 223, 226, 233, 220,. 215,. 217,. 224, 226,. 215. 217, 227, 224,. 220, 227,. 226, 224. 215, 226. 215,. 220,. 226,. 227, 227. 226,. 220. 227, 233,. 224,. 224. 224,. 226. 227,. 227,. 230, 220. 229, 226., 224, 224; 215, 227; 226, 229, 229,. 220; 220, 233; 224, 230, 230,. 217. 215. 224. 236, 227., 227, 230. 226. 226., 230, 229. 227. 230, 233. 236. 227., 220, 226; 227, 238, 233., 227,. 220, 230, 238,. 230,. 230, 233, 230, 227, 236, 230; 238, 236,. 227, 238, 238. & 230, 236. & 236, 233, 236, 236 (including the data for Qatar-4b) is a set of data that will be used to study exoplanets in our galactic neighbourhood. As a planet passes through the projected surface of the host star, the observed light curve will display a temporary reduction (a dip) in the overall flux from the planetary system. This level of reduction varies in di erent wavelength channels, which is key to revealing the chemical and dynamical properties of the planet's atmosphere."]

In [43]:
import re
pattern = r"\[.*?\]"

out = []
for name in text_list:
    if '[' in name and ']' in name:
      name = name[:name.find('[')] + name[name.find(']')+1:]
      out.append(name)
    print(out)
    cleaned_text = re.sub(" \d+", " ", str(out))
    cleaned_text = re.sub('[^A-Za-z0-9]+', " ", str(cleaned_text))
    print(cleaned_text)

["The Ariel Data Challenge  is a series of data competitions organised by mission scientists and engineers within the Ariel Consortium. The 2019 and 2021 editions of the challenge looked for innovative detrending solutions to extract planetary spectra from observed spectroscopic light curves. The simulations can however never represent the actual data, which will su er from unknown unknowns. Ensuring the reliable operation of Machine Learning models, as they move from simulated to real data, with unknowable ground truths, is of paramount importance. We conduct our experiments over the datasets from the 2019 and 2019 Ariel Data Challenges, hereby designated as ADC19 and ADC21 [9]. DRAFT version. Last modified: 2024-05-30 22:03:17Z  Transit light curves for the same star/planet system, produced by the simulation pipelines [12–15] of the ADC19 or ADC21 challenges. We treat the photon noise instances of the star’s first stellar spot configuration as repeated observations and aggregate them

In [10]:
import re

regex = [' \d+', ]

# Use re.sub() to remove the matched pattern
cleaned_text = re.sub(" \d+", " ", str(text_list))
cleaned_text = re.sub('[^A-Za-z0-9]+', " ", str(cleaned_text))
print(cleaned_text)

 The Ariel Data Challenge 9 11 is a series of data competitions organised by mission scientists and engineers within the Ariel Consortium The and editions of the challenge looked for innovative detrending solutions to extract planetary spectra from observed spectroscopic light curves The simulations can however never represent the actual data which will su er from unknown unknowns Ensuring the reliable operation of Machine Learning models as they move from simulated to real data with unknowable ground truths is of paramount importance We conduct our experiments over the datasets from the and Ariel Data Challenges hereby designated as ADC19 and ADC21 9 DRAFT version Last modified 05 30 03 17Z Transit light curves for the same star planet system produced by the simulation pipelines 12 15 of the ADC19 or ADC21 challenges We treat the photon noise instances of the star s first stellar spot configuration as repeated observations and aggregate them across the same system The data process sim

# Streamlit App

In [87]:
%%writefile app.py
import streamlit as st
import os, re
from transformers import BartForConditionalGeneration, BartTokenizer
from io import StringIO
import pandas as pd
import pymupdf
from datetime import datetime

# Intro Text
st.markdown('# PDF summarization using BART')
st.header('''PDF size must be below 200MB. Only 1 PDF per time, time will vary depending on size''')

input_file = st.file_uploader('Upload a PDF file')

# Save uploaded file
if input_file:
  with open(os.path.join(input_file.name),'wb') as f:
        f.write(input_file.getbuffer())
  st.success('Saved File')

st.markdown('# Summary')

# Load Model
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
# Load tokeniser
tokenizer = BartTokenizer.from_pretrained(model_name)

def pdf_to_dataframe(input_file):
  metadata = []
  pdf = pymupdf.open(input_file)
  # Get page content as dict and blocks of data
  for page in pdf:
    pages = page.get_text('dict')
    blocks = pages['blocks']
    # Extract metadata from blocks and filter for text, font size and font name
    for block in blocks:
      if 'lines' in block.keys():
        spans = block['lines']
        for span in spans:
          data = span['spans']
          for x in data:
            metadata.append((x['text'], x['size'], x['font']))
  pdf.close()
  output = pd.DataFrame(metadata,columns = ['text','size','font'])
  return output

if input_file:
  start = datetime.now()
  st.success('Summary loading, please wait...')
  page_wise_text = []
  # Kp-Regular holds actual text so filter for that
  df = pdf_to_dataframe(input_file)
  df = df.loc[df['font'] == 'Kp-Regular']
  corpus = ' '.join(df['text'])

  # Tokenise text
  tokens = tokenizer.encode(corpus, truncation=True, padding='longest', return_tensors='pt',max_length=1024)

  # Summarise text
  # summary = model.generate(tokens,max_length=1024, min_length=50, length_penalty=0.0, num_beams=4, early_stopping=True)
  summary = model.generate(tokens,max_length=1024, min_length=1000, length_penalty=0.0, num_beams=4, early_stopping=True)

  # Decode tokens and add to list
  decoded_output = tokenizer.decode(summary[0],skip_special_tokens=True)
  page_wise_text.append(decoded_output)
  # Remove '-'
  page_wise_text_cleaned = [ele.replace('- ', '') for ele in page_wise_text]
  # Remove brackets with numbers in them
  page_wise_text_cleaned = [re.sub(r'\[.*\]', ' ', x) for x in page_wise_text_cleaned]
  # Remove page/fig numbers and unwanted characters
  page_wise_text_cleaned = [re.sub(r' \d+', ' ', x) for x in page_wise_text_cleaned]
  page_wise_text_cleaned = [re.sub(r'[^A-Za-z0-9]+', ' ', x) for x in page_wise_text_cleaned]

  # Show Summary
  st.success('Summary ready!')
  st.text_area(label ="",value=page_wise_text_cleaned, placeholder="Please upload a PDF to get it's summary", height = 400)
  st.markdown('')
  end = datetime.now() - start

  st.header(end)

Overwriting app.py


In [21]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 1.621s


In [22]:
!curl ipv4.icanhazip.com

34.73.2.159


In [88]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

npx: installed 22 in 3.856s
your url is: https://huge-squids-call.loca.lt
^C
